In [31]:
%load_ext autoreload
%autoreload 2
from evo.dataset import FastaDataset
import torch

pfam_path  = "/homefs/home/lux70/storage/data/rocklinl.fasta"
ds = FastaDataset(pfam_path)
dl = torch.utils.data.DataLoader(ds, batch_size=1024)
batch = next(iter(dl))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 340M/340M [00:01<00:00, 339MB/s] 
100%|██████████| 340M/340M [00:02<00:00, 133MB/s] 


In [42]:
restypes = set([
    "A",
    "R",
    "N",
    "D",
    "C",
    "Q",
    "E",
    "G",
    "H",
    "I",
    "L",
    "K",
    "M",
    "F",
    "P",
    "S",
    "T",
    "W",
    "Y",
    "V",
])

In [51]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd

headers, seqs = batch
df = pd.DataFrame({"sequences": seqs})

def _protein_property(protein_sequence, property):
    protein_sequence = "".join(list(filter(lambda char: char in restypes, protein_sequence)))
    analyzer = ProteinAnalysis(protein_sequence)
    if property == "charge_at_pH":
        return getattr(analyzer, property)(pH=7)
    else:
        return getattr(analyzer, property)()



In [52]:
from copy import deepcopy
df1 = deepcopy(df)
df2 = deepcopy(df)
print(df.shape)
df

(1024, 1)


,sequences
0,MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVI...
1,MKAIAWLIVLTFLPEQVAWAVDYNLRGALHGAVAPLVSAATVATDG...
2,MEILGRNLPRILGNLVKTIKTAPVRVVARRGARTLTQKEFGKYLGS...
3,MSSQESPAVEFSTTTVSSVAVQAGDSKIVIAVIKCGKWIQLQLAES...
4,PPSFIHKPDPQEVLPGSNVKFTSVVTGTAPLKVSWFKGTTELVAGR...
...,...
1019,MSTQAPTFTQPLQSVVALEGSAATFEAQVSGSPVPEVSWFRDGQVL...
1020,MATQAPTFTQPLQSVVALEGSAATFEAQISGNPVPEVSWFRDGQVL...
1021,MSTQAPTFTQPLQSVVALEGSAATFEAQVSGNPVPEVSWFRDGQVL...
1022,MSTQAPTFTQPLQNVVALEGSAATFQAQVSGSPVPEVSWFRDGQVL...


In [55]:
%%time
# no multiprocessing

for property in properties:
    df1[property] = df.sequences.map(lambda seq: _protein_property(seq, property))

CPU times: user 24.1 s, sys: 10.2 ms, total: 24.1 s
Wall time: 24.1 s


In [56]:
import multiprocessing as mp
print(mp.cpu_count())

num_processes = min(mp.cpu_count(), df.shape[0]) 
print(num_processes)

from functools import partial

def process_chunk(chunk, prop):
    return chunk.sequences.map(lambda seq: _protein_property(seq, prop))

96
96


In [57]:
%%time
for prop in properties:  
    chunk_size = len(df) // num_processes  # Size of each chunk
    chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]
    
    pool = mp.Pool(processes=num_processes)
    fn = partial(process_chunk, prop=prop)
    results = pool.map(fn, chunks)
    pool.close()
    pool.join()
    df2[prop] = pd.concat(results)

CPU times: user 1.85 s, sys: 5.12 s, total: 6.97 s
Wall time: 13.3 s
